In [1]:
import spacy
import json
import os
from nltk.corpus import stopwords
import pandas as pd
import gensim
import gensim.corpora as corpora
import re
import contractions

In [2]:
files = []
subdir = tuple(os.walk("../imdb/imdb_movie_jsons/1990"))
for file_name in tuple(subdir)[0][-1]:
    with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
        files.append(json.load(f))

In [3]:
corpus = []
my_pos = set(["ADJ", "ADV",])
nlp = spacy.load("en_core_web_sm")
# nlp.max_length = 50_000_000
for file in files[10::]:
    content = [review["content"] for review in file["reviews"]]
    for review in content:
        string = nlp(review)
        clean = []
        for word in string:
            if word.pos_ in my_pos:
                clean.append(str(word.lemma_))
        corpus.append(" ".join(clean))
    # corpus = clean
    break

In [4]:
# corpus
# remove non english words again
# remove numbers
# remove words shorted than 3 letters
# remove punctuations

In [5]:
with open("../stopwords.json", "r") as f:
    common = set(json.load(f).extend(stopwords))
def clean(text: str):
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # expand contraction
    text = contractions.fix(text)
    # remove links
    text = re.sub(r"(ftp|http[s]?)://\S+", "", text)
    # remove punctuations
    sub_txt = r"[" + re.escape(string.punctuation) + r"]"
    text = re.sub(sub_txt, "", text)
    text = " ".join([word for word in text.split() if word not in common])

    return text.strip()

In [7]:
proc_data = [clean(string.split()) for string in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [8]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict)
vector = model[input_corpus]

In [9]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f
    # break

In [10]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(d.keys())

dict_keys(['More', 'able', 'again', 'alive', 'alone', 'already', 'apparently', 'appropriate', 'as', 'awful', 'back', 'bad', 'blind', 'bold', 'bona', 'cinematic', 'classic', 'daily', 'defective', 'emotionally', 'even', 'ever', 'exhausting', 'fide', 'first', 'fundamentally', 'good', 'hard', 'indeed', 'indelible', 'infamous', 'just', 'later', 'least', 'legged', 'local', 'long', 'low', 'magnum', 'majestic', 'many', 'mentally', 'modern', 'more', 'most', 'never', 'new', 'no', 'only', 'open', 'own', 'painful', 'particular', 'perfect', 'physically', 'pitiful', 'previous', 'qualified', 'rare', 'really', 'seriously', 'similar', 'simply', 'single', 'soft', 'special', 'still', 'straight', 'strongly', 'such', 'technically', 'that', 'truly', 'ultimately', 'unilaterally', 'vacant', 'various', 'vintage', 'well', 'woeful', 'worth', 'absolute', 'absolutely', 'anyways', 'complete', 'delightful', 'entire', 'fantastic', 'favorite', 'here', 'inept', 'kind', 'laugh', 'little', 'maybe', 'memorable', 'of', 'ol

In [11]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

                     TF-IDF
over                  0.968
trolling              0.925
cinimatic             0.881
sometimes             0.803
classically           0.776
mediocre              0.772
thrilled              0.767
compelling            0.744
tired                 0.733
ecstatic              0.725
messed                0.723
poorly                0.722
penchant              0.712
eatin                 0.710
passable              0.704
lacklustre            0.701
iconic                0.700
amateurish            0.682
seriously             0.675
comical               0.674
elite                 0.666
greatly               0.663
safe                  0.653
wobble                0.648
stuck                 0.648
excessively           0.639
unspeakable           0.634
reasonable            0.633
atrociously           0.625
subtext               0.622
fashionable           0.622
romantic              0.618
fun                   0.613
u're                  0.602
repulsive           